In [1]:
import os
import pandas as pd
import sys

In [2]:
# homey = os.path.abspath(os.path.dirname(__file__))
homey = os.getcwd()
sqlPath = os.path.join(homey, 'SQL')

In [3]:
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [4]:
# Using the Open SO Line (James' Report) query from FB_Simulator
myresults = connecttest.create_connection(sqlPath, 'Backlog_Report.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, sqlPath, 'Backlog_Report.xlsx')

Success!


In [5]:
# read the xlsx files into dataFrames
Backlog = os.path.join(sqlPath, 'Backlog_Report.xlsx')
SO = pd.read_excel(Backlog, header=0)

In [6]:
SO['CUSTOMER'] = SO['CUSTOMER'].str.replace('COMMAESCAPE', ',')
SO['DESCRIPTION'] = SO['DESCRIPTION'].str.replace('COMMAESCAPE', ',')
SO['NOTE'] = SO['NOTE'].str.replace('COMMAESCAPE', ',')

In [7]:
SO['PICK UP LOCATION'] = ''

In [8]:
for i in SO.index:
    if SO['GROUP'].at[i] in 'WA Tech Grp':
        SO['PICK UP LOCATION'].at[i] = 'WA Tech Grp'
    elif SO['GROUP'].at[i] in ['MD', 'TN', 'TX', 'Canada']:
        SO['PICK UP LOCATION'].at[i] = 'MD'
    elif SO['GROUP'].at[i] in ['CA N', 'CA LA', 'CO']:
        SO['PICK UP LOCATION'].at[i] = 'CA N'

In [9]:
SO['2nd PREFERENCE'] = ''

In [10]:
for i in SO.index:
    if SO['PICK UP LOCATION'].at[i] in 'WA Tech Grp':
        SO['2nd PREFERENCE'].at[i] = 'CA N'
    elif SO['PICK UP LOCATION'].at[i] in ['MD']:
        SO['2nd PREFERENCE'].at[i] = 'CA N'
    elif SO['PICK UP LOCATION'].at[i] in ['CA N']:
        SO['2nd PREFERENCE'].at[i] = 'WA Tech Grp'

In [11]:
SO['3RD PREFERENCE'] = ''

In [12]:
for i in SO.index:
    if SO['2nd PREFERENCE'].at[i] in 'WA Tech Grp':
        SO['3RD PREFERENCE'].at[i] = 'MD'
    elif SO['2nd PREFERENCE'].at[i] in 'CA N' and SO['PICK UP LOCATION'].at[i] in 'WA Tech Grp':
        SO['3RD PREFERENCE'].at[i] = 'MD'
    elif SO['2nd PREFERENCE'].at[i] in 'CA N' and SO['PICK UP LOCATION'].at[i] in 'MD':
        SO['3RD PREFERENCE'].at[i] = 'WA Tech Grp'

In [13]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(os.path.join(homey, 'Backlog_Transactions.xlsx'), engine='xlsxwriter')

In [14]:
# # Convert the dataframe to an XlsxWriter Excel object.
# SO.to_excel(writer, sheet_name='Backlog Report')

In [15]:
# Using the Remaining Inventory query from FB_Simulator
myresults = connecttest.create_connection(sqlPath, 'Inv_Qty_Summary.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, sqlPath, 'Inv_Qty_Summary.xlsx')

Success!


In [16]:
# read the xlsx files into dataFrames
Inventory = os.path.join(sqlPath, 'Inv_Qty_Summary.xlsx')
inv = pd.read_excel(Inventory, header=0)

In [17]:
inv['LOCATION'] = inv['LOCATION'].str.replace('COMMAESCAPE', ',')

In [18]:
# # Convert the dataframe to an XlsxWriter Excel object.
# inv.to_excel(writer, sheet_name='Inventory Summary')

In [19]:
SO['WA Tech Grp'] = ''

In [20]:
SO['CA N'] = ''

In [21]:
SO['MD'] = ''

In [22]:
SO['PURCHASE'] = ''

In [23]:
def find_qty_available(location, part, invSheet):
    partInv = invSheet[invSheet['PARTNUM'] == part].copy()
    partInv = partInv[partInv['GROUP'] == location].copy()
    if len(partInv) == 0:
        return(0, 'noIndex')
    else:
        return(partInv['QTY_AVAILABLE'].iat[0], partInv.index[0])

In [24]:
def pathfinder(locationGroup):
    print(locationGroup)
    if locationGroup == 'WA Tech Grp':
        locationPath = ['WA Tech Grp','CA N','MD','PURCHASE']
    if locationGroup == 'CA N':
        locationPath = ['CA N','WA Tech Grp','MD','PURCHASE']
    if locationGroup == 'MD':
        locationPath = ['MD','CA N','WA Tech Grp','PURCHASE']
    return(locationPath)

In [25]:
for i in SO.index:# for each line in SO
    thisPart = SO['PARTNUM'].at[i] # grab the part for reference
    thisLocation = SO['PICK UP LOCATION'].at[i] # grab the first priority location for reference

    LocationPriority = pathfinder(locationGroup = thisLocation) # find the appropriate order of locations
    qtyRemaining = SO['QTY'].at[i] # this is the quantity remaining to be allocated using inventory

    for eachLocation in LocationPriority: # rotate through the locations in order of priority
        if eachLocation == 'PURCHASE':
            SO['PURCHASE'].at[i] = qtyRemaining
            qtyRemaining = qtyRemaining - qtyRemaining

        if qtyRemaining > 0: # if there is inventory left to be allocated, keep running through location priorities
            qtyAvailable, j = find_qty_available(location=eachLocation, part=thisPart, invSheet=inv.copy())
            if qtyAvailable == 0:
                SO[eachLocation].at[i] = qtyAvailable
            
            elif qtyAvailable >= qtyRemaining:
                SO[eachLocation].at[i] = qtyRemaining
                qtyRemaining = qtyRemaining - SO[eachLocation].at[i]

                qtyAvailable = qtyAvailable - SO[eachLocation].at[i]
                inv['QTY_AVAILABLE'].at[j] = qtyAvailable # set inventory sheet line equal to qtyAvailable
            
            elif qtyAvailable < qtyRemaining:

                SO[eachLocation].at[i] = qtyAvailable
                qtyRemaining = qtyRemaining - SO[eachLocation].at[i]

                qtyAvailable = qtyAvailable - SO[eachLocation].at[i] # correct qtyAvailable
                inv['QTY_AVAILABLE'].at[j] = qtyAvailable # set inventory sheet line equal to qtyAvailable

        else:
            pass
#             SO[eachLocation].at[i] = 0

WA Tech Grp
WA Tech Grp
MD
MD
WA Tech Grp
WA Tech Grp
MD
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
CA N
MD
MD
MD
MD
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
CA N
MD
MD
MD
MD
MD
MD
MD
WA Tech Grp
WA Tech Grp
WA Tech Grp
MD
MD
CA N
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
CA N
CA N
CA N
CA N
WA Tech Grp
MD
MD
MD
MD
MD
MD
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
WA Tech Grp
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
WA Tech Grp
MD
CA N
CA N
CA N
MD
MD
MD
MD
CA N
CA N
CA N
CA N
CA N
CA N
MD
CA N
CA N
MD
MD
MD
MD
MD
MD
MD
WA Tech Grp
WA Tech Grp
WA Tech Grp
CA N
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
MD
CA N
CA N
CA N
CA N
MD
MD
MD
MD


In [26]:
SO

,SONUM,CUSTOMER,LINE,PARTNUM,DESCRIPTION,QTY,STATUS,DATE,DATEISSUED,CUSTOMER DELIVERY REQUEST,...,GROUP,WAREHOUSE,MAKEBUY,PICK UP LOCATION,2nd PREFERENCE,3RD PREFERENCE,WA Tech Grp,CA N,MD,PURCHASE
0,47811,"Comcast Cable (Hayward HE, CA)",65,050-004-10,Cable Rack; 20 in. (W); 2 in. (SB); 9' 8-1/2 (...,10.0,Entered,2017-04-03 00:00:00.0,2016-11-03 10:29:41.878,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,9,1,0
1,47811,"Comcast Cable (Hayward HE, CA)",122,840-017-10-1,Heat shrink; HSC34FR250 clear; 250' 0.75 id (2...,10.0,Partial,2017-04-03 00:00:00.0,2016-11-03 10:29:41.878,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,10,,,0
2,47769,"Comcast Cable (Germantown, TN)",47,050-533-10,Cable Rack; In-Line Splice Kit; 2 In. (SB); Ye...,10.0,Picked,2017-01-16 00:00:00.0,2016-12-19 11:21:00.068,NaN,...,MD,NaN,Buy,MD,CA N,WA Tech Grp,8,0,0,2
3,47769,"Comcast Cable (Germantown, TN)",78,850-071-10,Wire; hkup; flex #6 awg; blue w/ blk tracer,82.0,Entered,2017-01-13 21:00:00.0,2016-12-19 11:21:00.068,NaN,...,MD,NaN,Buy,MD,CA N,WA Tech Grp,,82,0,0
4,46155A,"Comcast Cable (Commerce City, CO)",20,161-200-10,TVSS; 1000A-4000A Alpha 277/480v; three-phase ...,2.0,Entered,2017-09-15 00:00:00.0,2017-05-05 09:36:40.237,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,0,0,2
5,46155A,"Comcast Cable (Commerce City, CO)",44,030-510-10,Phenolics; job specifics; black/white satin,1.0,Entered,2017-09-15 00:00:00.0,2017-05-05 09:36:40.237,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,0,0,1
6,49725,Comcast Cable (Martinsburg WV),8,010-560-10,Start up; Benning Invertronic Systems w/load bank,1.0,Entered,2017-06-01 00:00:00.0,2017-08-29 10:52:24.37,NaN,...,MD,NaN,Buy,MD,CA N,WA Tech Grp,0,0,0,1
7,47401A,"Comcast Cable (Novato Hub, CA)",66,050-985-10,Brace Foot; Channel 2in bar,40.0,Entered,2017-04-03 00:00:00.0,2017-09-20 11:20:34.595,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,0,0,40
8,47401A,"Comcast Cable (Novato Hub, CA)",112,050-098-10,Cable Rack; 12 x 10'; Gold; Seismic,10.0,Entered,2017-04-03 00:00:00.0,2017-09-20 11:20:34.595,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,0,0,10
9,47401A,"Comcast Cable (Novato Hub, CA)",113,050-614-10,Corner Kit Seismic for 2 Cable Rack Yellow Zinc,28.0,Entered,2017-04-03 00:00:00.0,2017-09-20 11:20:34.595,NaN,...,WA Tech Grp,NaN,Buy,WA Tech Grp,CA N,MD,0,0,20,8


In [27]:
# Convert the dataframe to an XlsxWriter Excel object.
SO.to_excel(writer, sheet_name='Backlog Report')

In [28]:
# Convert the dataframe to an XlsxWriter Excel object.
inv.to_excel(writer, sheet_name='Inventory Summary')

In [29]:
writer.save()